In [ ]:
def predictTextEmotion(text):
  tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", normalization=True)
  encoding = tokenizer.encode_plus(
    text,
    add_special_tokens=True,
    max_length=128,
    return_token_type_ids=False,
    return_attention_mask=True,
    return_tensors='pt',
    padding='max_length',
    truncation=True,
  )
  label_encoder = LabelEncoder()
  label_encoder.fit_transform(df_train.broadEmo)

  inputIds = encoding['input_ids'].flatten().unsqueeze(0)
  attentionMask = encoding['attention_mask'].flatten().unsqueeze(0)

  emotionScores = model(inputIds, attentionMask)
  emotion = torch.argmax(emotionScores)

  classProbabilities = {}
  with torch.no_grad():
    probs = torch.softmax(emotionScores, dim=1).flatten()
    for prob, label in zip(probs, label_encoder.classes_):
      classProbabilities[label] = "{:.2%}".format(prob.item())

  return label_encoder.inverse_transform([emotion.item()])[0], classProbabilities


In [ ]:
def computeModelPredictions(dataLoader):
  tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", normalization=True)
  groundTruth = torch.tensor([], dtype=torch.long)
  predictions = torch.tensor([], dtype=torch.long)
  i=0

  for x in dataLoader:
    print("{} out of 2459".format(i+32))
    i = i + 32
    inputIds, attentionMask, targets = x['input_ids'], x['attention_mask'], x['targets']

    emotionScores = model(inputIds.cuda(), attentionMask.cuda())
    predictions = torch.cat((predictions, emotionScores.argmax(dim=1).cpu()))
    groundTruth = torch.cat((groundTruth, targets.cpu()))

  return predictions, groundTruth

In [ ]:
model = TextClassifierModule.load_from_checkpoint("{}/models/emotion-recognizer.ckpt".format(os.getcwd()))
model.eval()
print(model)

In [ ]:
import json

text = input("Input text to get its emotion \n")
emotion, classProbabilities = predictTextEmotion(text)

print(emotion, "\n")
print(json.dumps(classProbabilities, indent=1))

In [ ]:
import json

text = input("Input text to get its emotion \n")
emotion, classProbabilities = predictTextEmotion(text)

print(emotion, "\n")
print(json.dumps(classProbabilities, indent=1))

In [ ]:
data_module.setup()
yHat, y = computeModelPredictions(data_module.test_dataloader())

In [ ]:
# Plot the confusion matrix

from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
labs = empDialogueData.broadEmo.unique().tolist()
labs.sort()

yList = y.tolist()
yHatList = yHat.tolist()
truth = list(map(lambda x: labs[x], yList))
preds = list(map(lambda x: labs[x], yHatList))

fig, ax = plt.subplots(figsize=(8, 8))
ConfusionMatrixDisplay.from_predictions(truth, preds, labels=labs, xticks_rotation=45, cmap="inferno", ax=ax)
plt.show()

# Explainability